<a href="https://colab.research.google.com/github/DonErnesto/masterclassSFI_2021/blob/main/notebooks/BreakoutSession_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Package installing and data import

!curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/data/X.csv.zip
!curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/data/y.csv.zip
!curl -O https://raw.githubusercontent.com/DonErnesto/masterclassSFI_2021/main/outlierutils.py


### Package imports 

We are using Python as a programming language. Its great advantage for data science purposes lies in the many and extensive open-source packages for data manipulation and machine learning. We will be using pandas for data handling, and scikit-learn (sklearn) for various outlier detection algorithms. 

Also, we imported a self-made module (outlierutils.py) that will be used for inspecting our results. 

In [ ]:
import pandas as pd

Next, we will load the data in a so-called DataFrame (a pandas object), and inspect it.

In [ ]:
X = pd.read_csv('X.csv.zip')
X.head()

The data describes credit card transactions, one transaction per row. Only the final column (Amount in USD) has a meaning. 

As you may notice, all features are numeric. They were compressed and anonymized using a mathematical operation called PCA. In reality, we always have to convert our data to a purely numerical form (however, we generally want to avoid losing touch of the meaning of the attributes, for instance reasons of explainability).

In this case, it is advantageous because no pre-processing or interpretation is needed, and we can feed the data directly into any algorithm, which will save us time. 

Before proceeding, let us determine the dimensions of the DataFrame:

In [ ]:
X.shape

In any realistic situation, we would not have access to labels (otherwise, we would be using a supervised approach) and typically know nothing about the fraction of positives. We will already give one fact away: the fraction of positive labels is 0.17%. 

## Assignment 1: Generate your own outlier score
We will generate an array with outlier scores, based on your own hand-made logic. 

#### Step 1: what shape should this array have? (# rows, # columns)

#### Step 2: using the .sum(), .max() and .abs() methods, create an outlier score, either by selecting one of the examples below, or by modifying them

#### Step 3: verify that the shape is correct


## Hints: 

1. we can select a single column by its name, and multiple columns by .iloc. 
Let's demonstrate with a smaller dataframe (the first 5 rows):


In [ ]:
small_df = X.head()
# A single column:
small_df['Amount']

In [ ]:
# All rows, and the first 5 columns:
small_df.iloc[:, :5]

In [ ]:
# All rows, all columns except the last 10 ones:
small_df.iloc[:, :-10]

We can use .max(axis=1) and .sum(axis=1) to get the max- and summation over all columns (this reduces the size of the dataframe from m rows x n columns to m rows. 

Also, we can use .abs() to convert the values to absolute (this doesn't change the size)

In [ ]:
small_df.iloc[:, :10].abs()

## Your experiments and work below:

In [ ]:
# Some examples to make an outlier score below. Uncomment (remove the "#") to execute it.
# Only the last executed one will be kept

homemade_outlier_score = X['Amount']
# homemade_outlier_score = X['V1'].abs()
# homemade_outlier_score = X.iloc[:, :10].abs().max(axis=1)




In [ ]:
# To verify the shape, add .shape to the dataframe and look at the output
homemade_outlier_score.shape

## Assignment 2 (10 minutes): Use an outlier algorithm to generate the outlier scores

We will use one of the various readily available outlier algorithms to generate scores. 

In Python, we typically first make an instance of a class (an object), than we perform various tasks (methods) with it. 

In [ ]:
# First, we import some algorithms 
# from sklearn.neighbors import LocalOutlierFactor
!pip install pyod==0.8.8
!pip install seaborn==0.11.1

from sklearn.covariance import EmpiricalCovariance #, MinCovDet # (MinCovDet may be very slow)
from pyod.models.knn import KNN
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import IsolationForest
from sklearn.mixture import GaussianMixture

#### Option: Nearest neighbours 
(uses a lot of time --> I may need to make the dataset smaller)

Let's create a NearestNeighbors object, and use that. First, we may want to read some documentation regarding the NearestNeighbors class:

In [ ]:
?NearestNeighbors

Most default settings seem ok for a start. An interesting parameter to change may however be n_neighbors.

Set n_neighbors to a value that seems okay (giving no arguments will get you all default values, as far as defaults are given)

In [ ]:
nn = NearestNeighbors(n_neighbors=10)

Now we have the object ready to accept data. We can directly fit it on the data using the .fit() method: 

In [ ]:
import time
t0 = time.process_time()
nn.fit(X.head(30000))
kneighbors = nn.kneighbors()[0]
duration = time.process_time() - t0
print(duration)

In [ ]:
kneighbors.mean(axis=1)

In [ ]:
y = pd.read_csv('y.csv.zip')

y.mean()*100

In [ ]:
y.head(30000).sum()

In [ ]:
!pip install seaborn
from outlierutils import plot_top_N, plot_outlier_scores

In [ ]:
cov_ = EmpiricalCovariance().fit(X)
# cov_ = MinCovDet().fit(X) # Robust estimation
mahalonobis_scores = cov_.mahalanobis(X)


In [ ]:
mahalonobis_scores = np.log10(mahalonobis_scores)
res = plot_outlier_scores(y.values, mahalonobis_scores, bw=0.1, title='Mahalonobis')

In [ ]:
res = plot_top_N(y.values, mahalonobis_scores, N=100)

In [ ]:
gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=1) # try also spherical
gmm.fit(X, )
gmm_scores = - gmm.score_samples(X)

In [ ]:
# gmm_scores = np.clip(gmm_scores, -15, 50)
res = plot_outlier_scores(y.values, np.log10(gmm_scores+100), bw=0.1, title='Pen digits, Mahalonobis (GMM)')

In [ ]:
res = plot_top_N(y.values, gmm_scores, N=100)

## Assignment 3: Plot and compare results
